# Importing Packages


In [1]:
import sklearn
import matplotlib.pyplot as plt
from sklearn import svm
import sklearn.model_selection as model_selection

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns; sns.set(font_scale = 1.2)
%matplotlib inline

In [3]:
import json
f = open('emojiclassify.json')
emojidata = json.load(f)
emojidata["emojis"]['😠']

'anger'

In [4]:
file = open("LikhithFinishedTags.txt","r")
data = file.readlines()
data[0]

'<tweet>\n'

In [5]:
tweets = []
for line in data:
  line = line.split()
  if len(line) > 0:
    if line[0] == '<tweet>':
      readtweet = True
      tweet = []
      continue
    elif line[0] == '</tweet>':
      readtweet = False

    if readtweet == True:
      if line[0] == "<emotion>" :
        tweets.append({'text':tweet,"emotion":line[1]})
      else:
        try:
          tweet.append({'word':line[0],'lang':line[1]})
        except:
          pass

# Emoji and Emoticon Feature 

In [6]:
pip install emoji

In [7]:
import emoji

def extract_emojis(s):
  return ''.join(c for c in s if c in emoji.UNICODE_EMOJI['en'])

In [8]:
all_features = []

def emoji_features(text):
  feature = [0,0,0]
  emojis_found = extract_emojis(text)
  for c in emojis_found:
    try:
      if emojidata["emojis"][c] == "happiness":
        feature[0] = feature[0] + 1
      elif emojidata["emojis"][c] == "sadness":
        feature[1] = feature[1] + 1
      elif emojidata["emojis"][c] == "anger":
        feature[2] = feature[2] + 1
    except:
      pass
  return feature

# Punctuation Feature

In [9]:
def punctuation_count(text):
  punctuation_num = 0
  for c in text:
    if c in ['!','?']:
      punctuation_num =  punctuation_num + 1

  return  punctuation_num

# Creating feature vector 


In [10]:
for tweet in tweets:
  othertext = ''
  for word in tweet['text']:
    if word['lang'] == 'OTH':
      othertext = othertext + word['word']

  feature = emoji_features(othertext)
  punctuation_num = punctuation_count(othertext)
  feature.append(punctuation_num)
  all_features.append(feature)


# Preprocessing



In [11]:
import re

In [12]:
tweetnum = 0
tweettextonly = []
for tweet in tweets:
  textonly = ''
  removewordlist = []
  for word in tweet['text']:
    if word['lang'] == 'OTH':
      removewordlist.append(word)
  for word in removewordlist:
    tweet['text'].remove(word)
  for word in tweet['text']:
    textonly = textonly + ' ' + word['word']
  tweettextonly.append(textonly)

In [13]:
 pip install git+https://github.com/siddharth17196/english-hindi-transliteration

  Cloning https://github.com/siddharth17196/english-hindi-transliteration to /tmp/pip-req-build-1sat5_h2
  Running command git clone -q https://github.com/siddharth17196/english-hindi-transliteration /tmp/pip-req-build-1sat5_h2


In [14]:
from elt import translit
to_hindi = translit('hindi')
to_hindi.convert(['1'])

['१']

In [15]:
for tweet in tweets:
  hintext = ''
  for word in tweet['text']:
    if word['lang'] == 'HIN':
      try:
        newword = to_hindi.convert([word['word']])
        word['word'] = newword[0]
      except:
        pass
tweets[0:10]

[{'emotion': 'Anger',
  'text': [{'lang': 'ENG', 'word': 'ban'},
   {'lang': 'HIN', 'word': 'होना'},
   {'lang': 'HIN', 'word': 'चाहिए'}]},
 {'emotion': 'Anger',
  'text': [{'lang': 'HIN', 'word': 'एक'},
   {'lang': 'HIN', 'word': 'इचछ'},
   {'lang': 'ENG', 'word': 'trophy'},
   {'lang': 'HIN', 'word': 'नहीं'},
   {'lang': 'HIN', 'word': 'जीते'},
   {'lang': 'ENG', 'word': 'under'},
   {'lang': 'ENG', 'word': 'his'},
   {'lang': 'ENG', 'word': 'captaincy'},
   {'lang': 'HIN', 'word': 'और'},
   {'lang': 'HIN', 'word': 'भी.'},
   {'lang': 'HIN', 'word': 'हारो'},
   {'lang': 'HIN', 'word': 'तो'},
   {'lang': 'HIN', 'word': 'हारो'},
   {'lang': 'HIN', 'word': 'उप्पेर'},
   {'lang': 'HIN', 'word': 'से'},
   {'lang': 'ENG', 'word': 'attitude'},
   {'lang': 'HIN', 'word': 'भी'},
   {'lang': 'HIN', 'word': 'सहो.'},
   {'lang': 'HIN', 'word': 'देश'},
   {'lang': 'ENG', 'word': 'ko'},
   {'lang': 'ENG', 'word': 'pal'},
   {'lang': 'HIN', 'word': 'रहा'},
   {'lang': 'HIN', 'word': 'है'},
   {'lan

# Uppercase


In [16]:
def uppercase_feature(text):
  splittweet = text.split()
  uppercase_words = 0
  for word in splittweet:   
    if word.upper() == word and len(word) > 1:
      uppercase_words = uppercase_words + 1
  return uppercase_words

In [17]:
tweetnum = 0
for tweet in tweettextonly:
  uppercase_words = uppercase_feature(tweet)
  all_features[tweetnum].append(uppercase_words)
  tweetnum = tweetnum + 1

# Repeated Charater Feature

In [18]:
def repeated_character_feature(text):
  splittweet = text.split()
  repeated_words = 0
  for word in splittweet:   
    first_word = ''
    second_word = ''
    for c in word:
      if c == first_word and first_word == second_word:
        repeated_words = repeated_words + 1
        break
      first_word = second_word
      second_word = c
  return repeated_words

In [19]:
tweetnum = 0
for tweet in tweettextonly:
  repeated_words = repeated_character_feature(tweet)
  all_features[tweetnum].append(repeated_words)
  tweetnum = tweetnum + 1

# Negation Words 

In [20]:
# importing english negated words
file = open("Negationwords.txt","r")
NegatedWords = file.read()
NegatedWordsList = NegatedWords.split()
NegatedWordsList

['never',
 'no',
 'nothing',
 'nowhere',
 'noone',
 'none',
 'not',
 'havent',
 'hasnt',
 'hadnt',
 'cant',
 'couldnt',
 'shouldnt',
 'wont',
 'wouldnt',
 'dont',
 'doesnt',
 'didnt',
 'isnt',
 'arent',
 'aint',
 'nahi',
 'nhi']

In [21]:
def negatedwords_feature(text):
  splittweet = text.split()
  negated_words = 0
  for word in splittweet:   
    if word.lower() in NegatedWordsList:
      negated_words = negated_words + 1
  return negated_words

In [22]:
tweetnum = 0
for tweet in tweettextonly:
  negated_words = negatedwords_feature(tweet)
  all_features[tweetnum].append(negated_words)
  tweetnum = tweetnum + 1

# Lexicon Emotion Feature

In [23]:
englexicon = pd.read_excel('NRC-Emotion-Lexicon.xlsx', sheet_name=None)
englexicon = englexicon['NRC-Lex-v0.92-word-translations']
englexicon.set_index("English (en)", inplace = True)
englexicon

,Hindi (hi),Positive,Negative,Anger,Disgust,Fear,Joy,Sadness,Surprise
English (en),,,,,,,,,
aback,अचंभे,0,0,0,0,0,0,0,0
abacus,अबेकस,0,0,0,0,0,0,0,0
abandon,छोड़ देना,0,1,0,0,1,0,1,0
abandoned,त्यागा हुआ,0,1,1,0,1,0,1,0
abandonment,संन्यास,0,1,1,0,1,0,1,1
...,...,...,...,...,...,...,...,...,...
zone,क्षेत्र,0,0,0,0,0,0,0,0
zoo,चिड़ियाघर,0,0,0,0,0,0,0,0
zoological,प्राणि,0,0,0,0,0,0,0,0


In [24]:
hinlexicon = pd.read_excel('NRC-Emotion-LexiconHindi.xlsx', sheet_name=None)
hinlexicon = hinlexicon['NRC-Lex-v0.92-word-translations']
hinlexicon.set_index("Hindi (hi)",inplace = True)

In [25]:
def lexicon_emotion_feature(text):
  tweetemotion = [0,0,0]
  for word in text['text']:
    try:
      if word['lang'] == 'ENG':
        wordemotion = englexicon.loc[word['word'].lower()]
      elif word['lang'] == 'HIN':
        wordemotion = hinlexicon.loc[word['word'].lower()]
      tweetemotion[0] = tweetemotion[0] + wordemotion['Joy']
      tweetemotion[1] = tweetemotion[1] + wordemotion['Sadness']
      tweetemotion[2] = tweetemotion[2] + wordemotion['Anger']
    except:
      pass
  return tweetemotion

In [26]:
tweetnum = 0
for tweet in tweets:
  tweetemotion = lexicon_emotion_feature(tweet)
  all_features[tweetnum] = all_features[tweetnum] + tweetemotion
  tweetnum = tweetnum + 1

# Intensifier Feature

In [27]:
# importing english intensifiers
file = open("IntensifierList.txt","r")
Intensifiers = file.read()
IntensifiersList = Intensifiers.split()

In [28]:
def Intensifiers_feature(text):
  splittweet = text.split()
  Intensifier_words = 0
  for word in splittweet:   
    if word.lower() in IntensifiersList:
      Intensifier_words = Intensifier_words + 1
  return Intensifier_words

In [29]:
tweetnum = 0
for tweet in tweettextonly:
  Intensifier_words = Intensifiers_feature(tweet)
  all_features[tweetnum].append(Intensifier_words)
  tweetnum = tweetnum + 1

# Word n-grams

In [30]:
import nltk
hindistopwords = []
hindistopwordfile= open("HindiStopWords.txt","r")
hindistopworddata = hindistopwordfile.readlines()
for line in hindistopworddata:
  line = line.split()
  hindistopwords.append(line[0])
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [31]:
import nltk
from sklearn.feature_extraction.text import CountVectorizer

num = 0
for tweet in tweettextonly:
  tweettextonly[num] = tweet.lower()
  num = num + 1 

stopwords = nltk.corpus.stopwords.words('english')
stopwords = stopwords + hindistopwords
# unigrams
cv = CountVectorizer(ngram_range=(1,1), stop_words = stopwords)
sparsearray = cv.fit_transform(tweettextonly)
vocab = cv.vocabulary_
count_values = sparsearray.toarray().sum(axis=0)

unigrams = []
num = 0
for ug_count, ug_text in sorted([(count_values[i],k) for k,i in vocab.items()], reverse=True):
    unigrams.append(ug_text)
    if(num == 50):
      break
    num = num + 1
print(unigrams)

#bigrams
cv = CountVectorizer(ngram_range=(2,2), stop_words = stopwords)
sparsearray = cv.fit_transform(tweettextonly)
vocab = cv.vocabulary_
count_values = sparsearray.toarray().sum(axis=0)

bigrams = []
num = 0
for bg_count, bg_text in sorted([(count_values[i],k) for k,i in vocab.items()], reverse=True):
    bigrams.append(bg_text)
    if(num == 50):
      break
    num = num + 1
print(bigrams)

#trigrams
cv = CountVectorizer(ngram_range=(3,3), stop_words = stopwords)
sparsearray = cv.fit_transform(tweettextonly)
vocab = cv.vocabulary_
count_values = sparsearray.toarray().sum(axis=0)

trigrams = []
num = 0
for tg_count, tg_text in sorted([(count_values[i],k) for k,i in vocab.items()], reverse=True):
    trigrams.append(tg_text)
    if(num == 50):
      break
    num = num + 1

print(trigrams)

print()

result = []
for tweet in tweettextonly:
    vec = []
    
    uni = []
    bi = []
    tri = []
    
    
    tokens = tweet.split(" ")
    
    # calculate the frequency of uni-gram
    for ug in unigrams:
        count = 0
        if ug in tweet:
            for token in tokens :
                if (ug == token):
                    count = count + 1
        uni.append(count)
    vec.append(uni)
    
    # calculate the frequency of bi-gram
    for bg in bigrams:
        count = 0
        if bg in tweet:
            for i in range(0, len(tokens)-1):
                bigram = tokens[i] + " " + tokens[i+1]
                if (bg == bigram):
                    count = count + 1
        bi.append(count)
    vec.append(bi)
    
    # calculate the frequency of tri-gram
    for tg in trigrams:
        count = 0
        if tg in tweet:
            for i in range(0, len(tokens)-2):
                trigram = tokens[i] + " " + tokens[i+1] + " " + tokens[i+2]
                if (tg == trigram):
                    count = count + 1
        tri.append(count)
    vec.append(tri)
    
    result.append(vec)

num = 0
for feature in all_features:
  all_features[num] = all_features[num] + result[num][0] + result[num][1] + result[num][2]
  num = num + 1


['sab', 'team', 'kya', 'bhai', 'tum', 'happy', 'nhi', 'mein', 'log', 'ipl', 'chahiye', 'main', 'le', 'international', 'desh', 'yeh', 'rehti', 'logo', 'liya', 'khelte', 'khelo', 'hoga', 'har', 'din', 'dil', 'dekh', 'bas', 'tournament', 'time', 'sad', 'saal', 'paise', 'nehi', 'mujhe', 'kr', 'khel', 'karo', 'hu', 'hamare', 'form', 'fir', 'baat', 'angry', 'ab', 'yar', 'waqt', 'wahi', 'vi', 'tujhe', 'toh', 'sirf']
['yeh ghour', 'yar wahi', 'wahi khilari', 'vi nah', 'tut geya', 'tum logoko', 'tum bachy', 'time mast', 'stalk kia', 'sab thhek', 'sab paise', 'sab hero', 'rehti tl', 'pichle year', 'paise khelte', 'nah harjaye', 'match day', 'mast rehti', 'maja alag', 'logoko dekhna', 'logo bura', 'kyonki isliye', 'kuchh khaas', 'kon ha', 'kisiko parva', 'khilari sab', 'khelte desh', 'khelo beta', 'khaas rehti', 'karo sab', 'jistaraka form', 'jao jaa', 'jaa kr', 'isliye dekne', 'ipl khelo', 'international tum', 'international cricket', 'hero dil', 'harjaye jistaraka', 'ha baap', 'ghour dekh', 'ge

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['kai'] not in stop_words.
  % sorted(inconsistent)


# Text Tagging

In [32]:
tag = []
for tweet in tweets:
  tag.append(tweet['emotion'])

# Tagging and Predicting

In [33]:
from sklearn import svm, datasets
import sklearn.model_selection as model_selection
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

X = all_features
y = tag
#poly = svm.SVC(kernel='poly', degree=3, C=1).fit(X, y)

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, train_size=0.80, test_size=0.20, random_state=101)
poly = svm.SVC(kernel='poly', degree=3, C=1).fit(X_train, y_train)
poly_pred = poly.predict(X_test)
poly_accuracy = accuracy_score(y_test, poly_pred)
poly_f1 = f1_score(y_test, poly_pred, average='weighted')
#print('Accuracy (Polynomial Kernel): ', "%.2f" % (poly_accuracy*100))
#print('F1 (Polynomial Kernel): ', "%.2f" % (poly_f1*100))

In [34]:
nltk.download('words')
words = set(nltk.corpus.words.words())

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [35]:
# Text Processing
def text_processing(text):
    re.sub("[^a-zA-z]","", text)
    
    return text

def n_grams(text) :
    result = []
    uni = []
    bi = []
    tri = []
    
    tokens = text.split(" ")
    
    # calculate the frequency of uni-gram
    for ug in unigrams:
        count = 0
        if ug in text:
            for token in tokens :
                if (ug == token):
                    count = count + 1
        uni.append(count)
    result.append(uni)
    
    # calculate the frequency of bi-gram
    for bg in bigrams:
        count = 0
        if bg in text:
            for i in range(0, len(tokens)-1):
                bigram = tokens[i] + " " + tokens[i+1]
                if (bg == bigram):
                    count = count + 1
        bi.append(count)
    result.append(bi)
    
    # calculate the frequency of tri-gram
    for tg in trigrams:
        count = 0
        if tg in text:
            for i in range(0, len(tokens)-2):
                trigram = tokens[i] + " " + tokens[i+1] + " " + tokens[i+2]
                if (tg == trigram):
                    count = count + 1
        tri.append(count)
    result.append(tri)
    
    return result

def lexicon_emoticon_features(text):
    emotionvector = [0,0,0]
    sent = text
    sent = " ".join(w for w in nltk.wordpunct_tokenize(sent) if w.lower() in words or not w.isalpha())
    sent = sent.split()
    
    for word in text.split():
        try:
            if word in sent:
                wordemotion = englexicon.loc[word.lower()]
            else:
                newword = to_hindi.convert([word])
                newword = newword[0]
                wordemotion = hinlexicon.loc[newword]
  
            emotionvector[0] = emotionvector[0] + wordemotion['Joy']
            emotionvector[1] = emotionvector[1] + wordemotion['Sadness']
            emotionvector[2] = emotionvector[2] + wordemotion['Anger']
        
        except:
            pass
        
    return emotionvector

In [36]:
for i in range(1,20):
  txt = input("Enter the sentence : ")

  # Vectorization of all features
  features = []

  features = emoji_features(txt)

  features.append(punctuation_count(txt))

  txt = text_processing(txt)

  features.append(uppercase_feature(txt))
  features.append(repeated_character_feature(txt))
  features.append(negatedwords_feature(txt))
  features = features + lexicon_emoticon_features(txt)
  features.append(Intensifiers_feature(txt))
  newlist = n_grams(txt)
  features = features + newlist[0] + newlist[1] + newlist[2]

  poly_pred = poly.predict([features])
  print(poly_pred[0])

Enter the sentence : i am happy 😂😂
Happiness
Enter the sentence : i am crying
Sadness
Enter the sentence : i am crying 😂😂
Happiness
Enter the sentence : i am sad
Sadness
Enter the sentence : main udaas hu
Sadness
Enter the sentence :  he passed away
Sadness
Enter the sentence : he failed the exam
Sadness
Enter the sentence : he passed the exam
Sadness
Enter the sentence : STOP THIS PLISS !!
Anger
Enter the sentence : bann ipl 😠
Anger
Enter the sentence : ban ipl
Sadness
Enter the sentence : i wish you a very happy happy birthday
Happiness
Enter the sentence : i can't take this anymore
Sadness
Enter the sentence : I AM EXCITED
Anger
Enter the sentence : help me pls
Sadness
Enter the sentence : i am disappointed
Sadness
Enter the sentence : i am frustrated
Sadness
Enter the sentence : KAL MERA EXAM HAI i don't have anytime FOR NONSENSE
Anger
Enter the sentence : kyu mujhe DISTURB kar rahe ho
Anger
